In [171]:
import pandas as pd
from collections import Counter
import re

df = pd.read_csv('../../data/augmented/aya-32b.csv')
df

,ID,text,target
0,start,start,start
1,augmentation,"1) '코로나19' 장기화, 가정폭력 신고 전년比 증가\nanswer: 2) AI,...",-1
2,augmentation,"1) 북한, 새로운 우주 발사 계획 발표 \n2) 서울 지하철, AI 기반 시스템 ...",-1
3,augmentation,"\nAI, 인간 예술가와 협업 전시회 개최\n정부, 전 국민 기본소득제 시범 도입\...",-1
4,augmentation,"1) 북극권, 기후변화로 영구동토층 해동 위기\n북극권의 영원한 얼음, 해동으로 생...",-1
...,...,...,...
246,augmentation,"1) 북한, 새로운 우주 발사체 시험 성공\n남: 2) K팝 아이돌, 미국 빌보드 ...",-1
247,augmentation,"1) AI, 미술 경매 기록 경신\n진행자: 좋아, 그럼 다음으로 넘어갈게. 최근에...",-1
248,augmentation,"1) AI, 인간과 창의력 대결서 승리\n중3) K팝 아이돌, 국제 음악 차트 2주...",-1
249,augmentation,"\n북한, 새로운 우주 개발 계획 발표: ""별하늘을 향한 큰 발걸음""\n스마트폰 중...",-1


In [172]:
# 1행 제거
df = df.drop(0)

# 'text' 열의 데이터를 '\n' 기준으로 분리하여 새로운 행으로 확장
df = df.set_index(['ID', 'target']).text.str.split('\n', expand=True).stack().reset_index(level=2, drop=True).reset_index()
df.columns = ['ID', 'target', 'text']
df = df[['ID', 'text', 'target']]

print(f'개행 기준 분리 행 수: {len(df)}')

# empty string 제거
df = df[df['text'] != '']
print(f'Empty string 제거 후 행 수: {len(df)}')

# 텍스트 길이 기반 제거
df['text_len'] = df['text'].str.len()
df = df[df['text_len'] >= 15]
df = df[df['text_len'] < 40]
print(f'텍스트 길이 기반 제거 후 행 수: {len(df)}')
df

개행 기준 분리 행 수: 7458
Empty string 제거 후 행 수: 6180
텍스트 길이 기반 제거 후 행 수: 4044


,ID,text,target,text_len
0,augmentation,"1) '코로나19' 장기화, 가정폭력 신고 전년比 증가",-1,30
1,augmentation,"answer: 2) AI, 한국 예술계 '창작의 패러다임' 바꾸다",-1,36
3,augmentation,)3) 국내 첫 '수중 레스토랑' 개장…바다 속 식사 경험,-1,32
4,augmentation,Answer:4) 서울 도심에 '미니 숲' 조성…미세먼지 저감 기대,-1,37
5,augmentation,")5) 아이돌 그룹 멤버, 음주운전 적발…팀 활동 중단 위기",-1,34
...,...,...,...,...
7453,augmentation,"Z세대, 패션 트랜드 역주행 주도",-1,18
7454,augmentation,"수능 대체 시험, 공정성 논란 확산",-1,19
7455,augmentation,"기후변화, 북극 북 극곰 멸종 위기 가속화",-1,24
7456,augmentation,"AI 아티스트, 미술계 판도 뒤흔드나",-1,20


In [173]:
remove_words = ['주제', '글자', '제목', '독자', '어조', '답변', '과제', '아이디어', '여러분', '설명', '당신', '?', '!']

# 단어들이 포함된 행 제거
df = df[~df['text'].apply(lambda x: any(word in x for word in remove_words))]
print(f'removed_words 단어들이 포함된 행 제거 후 행 수: {len(df)}')

# 새로운 열에 한국어 비율 추가
def korean_ratio(text):
    korean_count = len(re.findall(r'[가-힣]', text))
    total_count = len(text)
    if total_count == 0:
        return 0
    return korean_count / total_count

df['korean_ratio'] = df['text'].apply(korean_ratio)
df = df[df['korean_ratio'] > 0.4]
print(f'한국어 비율 추가 후 행 수: {len(df)}')
df

# 'text' 열의 데이터에서 'answer' 또는 'Answer' 또는 '답:' 문자열 제거
df['text'] = df['text'].apply(lambda x: re.sub(r'\b(answer|Answer|답:)\b', '', x).strip())
df

removed_words 단어들이 포함된 행 제거 후 행 수: 3696
한국어 비율 추가 후 행 수: 3411


/tmp/ipykernel_2846739/2605188443.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['korean_ratio'] = df['text'].apply(korean_ratio)


,ID,text,target,text_len,korean_ratio
0,augmentation,"1) '코로나19' 장기화, 가정폭력 신고 전년比 증가",-1,30,0.533333
1,augmentation,": 2) AI, 한국 예술계 '창작의 패러다임' 바꾸다",-1,36,0.416667
3,augmentation,)3) 국내 첫 '수중 레스토랑' 개장…바다 속 식사 경험,-1,32,0.562500
4,augmentation,:4) 서울 도심에 '미니 숲' 조성…미세먼지 저감 기대,-1,37,0.486486
5,augmentation,")5) 아이돌 그룹 멤버, 음주운전 적발…팀 활동 중단 위기",-1,34,0.588235
...,...,...,...,...,...
7453,augmentation,"Z세대, 패션 트랜드 역주행 주도",-1,18,0.666667
7454,augmentation,"수능 대체 시험, 공정성 논란 확산",-1,19,0.684211
7455,augmentation,"기후변화, 북극 북 극곰 멸종 위기 가속화",-1,24,0.666667
7456,augmentation,"AI 아티스트, 미술계 판도 뒤흔드나",-1,20,0.650000


In [174]:
df['text'] = df['text'].apply(lambda x: ''.join([char if re.match(r'[가-힣a-zA-Z0-9\s,.\'\"]', char) else '' for char in x]))
df = df.drop(columns=['text_len', 'korean_ratio'])
df.to_csv('../../data/augmented/aya-32b-cleaned.csv', index=False)